In [ ]:
!nvidia-smi

Fri May 30 20:26:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!pip install --upgrade cmake==3.22.5

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,719 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,953 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubunt

In [ ]:
!git clone --recursive https://github.com/xtinkt/instant-ngp-video
%cd instant-ngp-video

Cloning into 'instant-ngp'...
remote: Enumerating objects: 4372, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 4372 (delta 17), reused 16 (delta 16), pack-reused 4337 (from 3)
Receiving objects: 100% (4372/4372), 187.06 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (2788/2788), done.
Submodule 'dependencies/OpenXR-SDK' (https://github.com/KhronosGroup/OpenXR-SDK.git) registered for path 'dependencies/OpenXR-SDK'
Submodule 'dependencies/args' (https://github.com/Taywee/args) registered for path 'dependencies/args'
Submodule 'dependencies/dlss' (https://github.com/NVIDIA/DLSS) registered for path 'dependencies/dlss'
Submodule 'dependencies/glfw' (https://github.com/Tom94/glfw) registered for path 'dependencies/glfw'
Submodule 'dependencies/imgui' (https://github.com/ocornut/imgui.git) registered for path 'dependencies/imgui'
Submodule 'dependencies/pybind11' (https://github.com/Tom94/pybind11) registered for path 'dep

In [ ]:
compute_capability = "80" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability


env: TCNN_CUDA_ARCHITECTURES=80


In [ ]:
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH

Using FullyFusedMLP
env: NN_CONFIG_PATH=./configs/nerf/base.json


In [ ]:
# !export CUDA_HOME=/usr/local/cuda
# !export PATH="/usr/local/cuda/bin:$PATH"
# !export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/usr/local/cuda/lib64"
# !export LIBRARY_PATH="$LIBRARY_PATH:/usr/local/cuda/lib64"

In [ ]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.5.82
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- No release type specified. Setting to 'Release'.
-- Obtained CUDA architectures from environment variable TCNN_CUDA_ARCHITECTURES=80
-- Targeting CUDA architectures: 80
-- Module support is disabled.
-- Version:

In [ ]:
!cmake --build build --config RelWithDebInfo -j `nproc`

[  2%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/pathescape.ptx
[  5%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raystab.ptx
[ 10%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/format.cc.o
[ 10%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raytrace.ptx
[ 13%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/os.cc.o
[ 13%] Built target optix_program
[ 15%] Linking CXX static library libfmt.a
[ 15%] Built target fmt
[ 18%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/common_host.cu.o
[ 23%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cpp_api.cu.o
[ 23%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cutlass_mlp.cu.o
[ 26%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/encoding.cu.o
[ 28%] Building CUDA object dependencies/tiny-c

In [ ]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 22.9 MB/s eta 0:00:00
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12077 sha256=9a24b9cca84749965dfac70be18f73ee0908cfa651311acd194da097c412d536
  Stored in directory: /root/.cache/pip/wheels/f2/cb/58/77f1f0b3f5fa6eb3335cb0cb73de4d4bef93489dc3a3373ae8
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=9f2186810a50d70444db68d451860a64347895a060ee4cb6bf64ab91dc34e9c3
  Stored in directory: /root/.cache/pip/wheels/6c/c7/01/d277a7a673ffc81a1e71e816c2e5bc5f1602f9e8192c25d21c
Successfully built commentjson lark-parser


Mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
scene_path = "/content/drive/MyDrive/kek_v" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

In [ ]:
!ls /content/drive/MyDrive/kek_v

base_cam.json  frame_21  frame_34  frame_47  frame_6   frame_72
frame_1        frame_22  frame_35  frame_48  frame_60  frame_73
frame_10       frame_23  frame_36  frame_49  frame_61  frame_74
frame_11       frame_24  frame_37  frame_5   frame_62  frame_75
frame_12       frame_25  frame_38  frame_50  frame_63  frame_76
frame_13       frame_26  frame_39  frame_51  frame_64  frame_77
frame_14       frame_27  frame_4   frame_52  frame_65  frame_78
frame_15       frame_28  frame_40  frame_53  frame_66  frame_79
frame_16       frame_29  frame_41  frame_54  frame_67  frame_8
frame_17       frame_3	 frame_42  frame_55  frame_68  frame_80
frame_18       frame_30  frame_43  frame_56  frame_69  frame_81
frame_19       frame_31  frame_44  frame_57  frame_7   frame_9
frame_2        frame_32  frame_45  frame_58  frame_70  transforms.json
frame_20       frame_33  frame_46  frame_59  frame_71


In [ ]:
from tqdm import tqdm

for i in tqdm(range(1, 80)):
  scene_path = "/content/drive/MyDrive/kek_v/frame_{}".format(i)
  train_steps = 5000
  snapshot_path = os.path.join(scene_path, f"{train_steps}.ingp")
  !python ./scripts/run.py --scene {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path}

  0%|          | 0/79 [00:00<?, ?it/s]

20:44:06 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: NVIDIA A100-SXM4-40GB [80]
20:44:08 INFO     Loading NeRF dataset from
20:44:09 INFO       /content/drive/MyDrive/kek_v/frame_1/transforms.json
20:44:09 WARNING    /content/drive/MyDrive/kek_v/frame_1/base_cam.json does not contain any frames. Skipping.
20:44:09 PROGRESS []   0% ( 0/14)  0s/inf20:44:10 PROGRESS []   7% ( 1/14) 0s/7s20:44:10 PROGRESS []  14% ( 2/14) 0s/4s20:44:10 PROGRESS []  21% ( 3/14) 0s/2s20:44:10 PROGRESS []  29% ( 4/14) 0s/2s20:44:10 PROGRESS []  36% ( 5/14) 0s/2s20:44:10 PROGRESS []  43% ( 6/14) 1s/2s20:44:10 PROGRESS []  50% ( 7/14) 1s/2s20:44:11 PROGRESS []  57% ( 8/14) 1s/2s20:44:11 PROGRESS []  64% ( 9/14) 1s/2s20:44:11 PROGRESS []  71% (10/14) 1s/2s20:44:11 PROGRESS []  79% (11/14) 1s/2s20:44:11 PROGRESS []  86% (12/14) 1s/2s20:44:11 PROGRESS []  93% (13/14) 2s/2s20:44:11 PROGRESS [] 100% (14/14) 2s/2s20:44:11 SUCCESS  Loaded 14 images after 2s
20:44:11 INFO       cam_aabb=[min=[1.48711,-0.139681,0.2

  1%|▏         | 1/79 [02:12<2:52:17, 132.53s/it]

^C
20:46:19 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: NVIDIA A100-SXM4-40GB [80]
20:46:19 INFO     Loading NeRF dataset from
20:46:20 INFO       /content/drive/MyDrive/kek_v/frame_2/transforms.json
20:46:20 WARNING    /content/drive/MyDrive/kek_v/frame_2/base_cam.json does not contain any frames. Skipping.
20:46:20 PROGRESS []   0% ( 0/14)  0s/inf20:46:21 PROGRESS []   7% ( 1/14) 0s/8s20:46:21 PROGRESS []  14% ( 2/14) 0s/4s20:46:21 PROGRESS []  21% ( 3/14) 0s/3s20:46:21 PROGRESS []  29% ( 4/14) 0s/2s20:46:21 PROGRESS []  36% ( 5/14) 0s/2s20:46:21 PROGRESS []  43% ( 6/14) 0s/2s20:46:21 PROGRESS []  50% ( 7/14) 1s/2s20:46:22 PROGRESS []  57% ( 8/14) 1s/2s20:46:22 PROGRESS []  64% ( 9/14) 1s/2s20:46:22 PROGRESS []  71% (10/14) 1s/2s20:46:22 PROGRESS []  79% (11/14) 1s/2s20:46:22 PROGRESS []  86% (12/14) 1s/1s20:46:22 PROGRESS []  93% (13/14) 1s/1s20:46:22 PROGRESS [] 100% (14/14) 1s/1s20:46:22 SUCCESS  Loaded 14 images after 1s
20:46:22 INFO       cam_aabb=[min=[1.48711,-0.139681,

In [ ]:
import os
output_video_path = os.path.join("/content/drive/MyDrive/kek_v/output_video_5000_compile.mp4")
print(output_video_path)

!python scripts/run_multiple.py --load_snapshot {snapshot_path} --model_subpath "5000.ingp" \
    --video_camera_path "/content/drive/MyDrive/kek_v/frame_10/base_cam.json" --video_n_seconds 3 \
    --video_fps 2 --video_output "/content/drive/MyDrive/kek_v/output_video_5000_compile.mp4"
print(f"Generated video saved to:\n{output_video_path}")

/content/drive/MyDrive/kek_v/output_video_5000_compile.mp4
Traceback (most recent call last):
  File "/content/instant-ngp/scripts/run_multiple.py", line 25, in <module>
    import pyngp as ngp # noqa
    ^^^^^^^^^^^^^^^^^^^
ImportError: /content/instant-ngp/build/pyngp.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3ngp7Testbed16save_raw_volumesERKN10filesystem4pathEiNS_11BoundingBoxEb
Generated video saved to:
/content/drive/MyDrive/kek_v/output_video_5000_compile.mp4


In [ ]:
cd ..

/content/instant-ngp


In [ ]:
!ls tmp

/content/instant-ngp/instant-ngp


In [ ]:
ls /content/drive/MyDrive/xtinkt/instant-ngp-video/artems/

frame_0/   frame_14/  frame_2/   frame_25/  frame_30/  frame_36/  frame_41/  frame_47/
frame_1/   frame_15/  frame_20/  frame_26/  frame_31/  frame_37/  frame_42/  frame_5/
frame_10/  frame_16/  frame_21/  frame_27/  frame_32/  frame_38/  frame_43/  frame_6/
frame_11/  frame_17/  frame_22/  frame_28/  frame_33/  frame_39/  frame_44/  frame_7/
frame_12/  frame_18/  frame_23/  frame_29/  frame_34/  frame_4/   frame_45/  frame_8/
frame_13/  frame_19/  frame_24/  frame_3/   frame_35/  frame_40/  frame_46/  frame_9/


In [ ]:
import torch, gc

In [ ]:
torch.cuda.empty_cache()
gc.collect()

34